## APPLICAZIONE SUL TEST SET DELLE MODIFICHE FATTE SUL TRAINING SET

In [1]:
import pandas as pd
from constants import*
from constants import numeric_columns as num_cols
import numpy as np
import ast

test_df = pd.read_csv("test.csv")

### Trasformazione \N di genres in NaN

In [2]:
#trasforma valori \N di genres in nan
test_df.loc[test_df["genres"] == r'\N', "genres"] = np.nan

### Trasformazione dei generi in liste di stringhe; per i record NaN vengono trasformati in [""]

In [3]:
# Replace NaN values with an empty string
test_df["genres"] = test_df["genres"].fillna("")

# Split genres into lists
test_df["genres"] = test_df["genres"].str.split(',')

# Debug: Check the data types within the lists
print(test_df['genres'].apply(lambda x: [type(genre) for genre in (x if isinstance(x, list) else [])]))

# Preview the updated genres column
print(test_df["genres"].head())

0                      [<class 'str'>, <class 'str'>]
1                                     [<class 'str'>]
2                                     [<class 'str'>]
3                                     [<class 'str'>]
4       [<class 'str'>, <class 'str'>, <class 'str'>]
                            ...                      
5473                                  [<class 'str'>]
5474    [<class 'str'>, <class 'str'>, <class 'str'>]
5475                                  [<class 'str'>]
5476                                  [<class 'str'>]
5477                                  [<class 'str'>]
Name: genres, Length: 5478, dtype: object
0             [Music, Short]
1                   [Horror]
2               [Reality-TV]
3                [Game-Show]
4    [Crime, Drama, Mystery]
Name: genres, dtype: object


### Trasformare i valori \N in nan nelle colonne numeriche

In [4]:
#uniformo assegnando NaN a tutti i valori nulli
for col in num_cols:
    #Converts the values in column col to numeric types (e.g., int, float). 
    # #If a value cannot be converted  (e.g., invalid strings), it is replaced with NaN (missing value).
    test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

    #abbiamo trasformato i valori numerici in interi e non float (con Int64 accetta i Nan)
    #test_df[col] = test_df[col].astype('Int64')

### Ceiling per i rating

In [5]:
#ceiling per rating
test_df['rating'] = test_df['rating'].str.extract(
        r",\s*(\d+)\]"
    ).astype(int)

### CountryofOrigin come lista di stringhe

In [6]:
#traformazione paesi in liste di stringhe
test_df['countryOfOrigin'] = test_df['countryOfOrigin'].apply(ast.literal_eval)

print(test_df['countryOfOrigin'].apply(lambda x: [type(country) for country in x]))

0       [<class 'str'>]
1       [<class 'str'>]
2       [<class 'str'>]
3       [<class 'str'>]
4       [<class 'str'>]
             ...       
5473    [<class 'str'>]
5474    [<class 'str'>]
5475    [<class 'str'>]
5476    [<class 'str'>]
5477    [<class 'str'>]
Name: countryOfOrigin, Length: 5478, dtype: object


### GESTIONE 1 MISSING VALUES --> trasformazione awardWins a 0 per i nan

In [7]:
test_df["awardWins"] = test_df["awardWins"].fillna(0)

### GESTIONE 2 MISSING VALUES --> one-hot per generi; i missing hanno 0 in tutte le variabili

In [8]:
# Flatten all lists and get unique genres
all_genres = set(genre for sublist in test_df['genres'] if isinstance(sublist, list) for genre in sublist)
print(all_genres)

{'', 'Western', 'Thriller', 'Horror', 'Animation', 'Family', 'News', 'Sport', 'Reality-TV', 'Biography', 'Sci-Fi', 'Fantasy', 'Documentary', 'Talk-Show', 'Drama', 'Music', 'Game-Show', 'Adventure', 'Romance', 'Mystery', 'Short', 'Action', 'Adult', 'History', 'Musical', 'Crime', 'Film-Noir', 'War', 'Comedy'}


In [9]:
# Create a one-hot encoded column for each genre
for genre in all_genres:
    test_df[f"is_{genre}"] = test_df['genres'].apply(lambda x: 1 if genre in x else 0)

In [10]:
#droppiamo colonna ["is_"] così che il vettore per i record che non hanno genere siano tutti 0
test_df.drop(["is_"], axis=1, inplace=True)

### GESTIONE 3 MISSING VALUES --> sostituire i nan di runtimeMins come mediana raggruppato per titletype

In [11]:
#creiamo nuova colonna runtimeMinutes dove sostituiamo i nan con mediana 
test_df["fill_runtimeMinutes"] = test_df.groupby(["titleType"])["runtimeMinutes"].transform(lambda x: x.fillna(x.median()))

### Drop della colonna "genres" dato che abbiamo già fatto il one-hot encoding

In [12]:
test_df.drop(["genres"], axis=1, inplace=True)

### Drop colonne redundant

In [13]:
test_df.drop(["bestRating"], axis=1, inplace=True)
test_df.drop(["worstRating"], axis=1, inplace=True)
test_df.drop(["isRatable"], axis=1, inplace=True)
test_df.drop(["ratingCount"], axis=1, inplace=True)

### Applicazione delle modifiche al nuovo csv

In [14]:
test_df.to_csv('pp_test.csv', index=False, header=True)